In [ ]:
!pip install pettingzoo[mpe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 799 kB 6.4 MB/s 
     |████████████████████████████████| 836 kB 34.6 MB/s 
     |████████████████████████████████| 18.3 MB 1.4 MB/s 


In [ ]:
from pettingzoo.mpe import simple_adversary_v2
env = simple_adversary_v2.env(N=2,max_cycles = 25 , continuous_actions = False)
env.reset()

In [ ]:
import numpy as np

class Replay():
  def __init__(self,max_size,n_actions,input_shape):
    self.mem_size = max_size
    self.mem_counter = 0
    self.state_memory = np.zeros((self.mem_size,input_shape))
    self.new_state_memory = np.zeros((self.mem_size,input_shape))
    self.action_memory = np.zeros((self.mem_size,n_actions))
    self.reward_memory = np.zeros(self.mem_size)
    self.terminal_memory = np.zeros(self.mem_size,dtype = bool)

  def store_trans(self,state,action,reward,done):
    index = self.mem_counter % self.mem_size
    self.state_memory[index] = state
    self.action_memory[index] = action
    self.reward_memory[index] = reward
   
    self.terminal_memory[index] = done
    if (self.mem_counter > 0):
       self.new_state_memory[index-1] = state

    self.mem_counter += 1

  def sample_buffer(self,batch):
    self.batch = batch
    states = self.state_memory[batch]
    actions = self.action_memory[batch]
    rewards = self.reward_memory[batch]
    states_ = self.new_state_memory[batch]
    dones = self.terminal_memory[batch]
    
    return   states,actions,rewards,states_,dones

  def returnSample(self,batch_size):
    max_mem = min(batch_size,self.mem_counter)
    batch = np.random.choice(max_mem,batch_size) 
    return batch

  def returnshit(self):
    return self.state_memory

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense

class CriticNet(keras.Model):
  def __init__(self,n_actions, dim_layer):
    super(CriticNet,self).__init__()
    self.dim_size = dim_layer
    self.n_actions = n_actions
    self.dim_size = 128
    self.layer_one = Dense(self.dim_size,activation='relu')
    self.layer_two = Dense(self.dim_size,activation='relu')
    self.q_value = Dense(1,activation=None)

  def call(self,inputs):
    states,actions = inputs
    input_thing = tf.concat([states, actions],axis = 1)
    action_value = self.layer_one(input_thing)
    action_value = self.layer_two(action_value)

    q = self.q_value(action_value)

    return q

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense

class ActorNet(keras.Model):
  def __init__(self,n_actions, dim_layer):
    super(ActorNet,self).__init__()
    self.dim_size = dim_layer
    self.n_actions = n_actions

    self.layer_one = Dense(self.dim_size, activation = 'relu')
    self.layer_two = Dense(self.dim_size, activation = 'relu')
    self.policy = Dense(self.n_actions, activation = 'softmax')

  def call(self,state):
    probs = self.layer_one(state)
    probs = self.layer_two(probs)

    mu = self.policy(probs)
    return mu

In [ ]:
class Agent_Good:
  def __init__(self,input_dims,alpha = 0.01,beta =1e-5 , env = None ,
               gamma = 0.95, n_actions = 5, max_size = 10000, tau = 0.001,
               layer_size = 256, batch_size = 50,noise = 0.1, name = None):
    
    self.gamma = gamma
    self.tau = tau
    self.memory = Replay(max_size,n_actions,input_dims)
    self.batch_size = batch_size
    self.n_actions = n_actions
    self.act_name = name+' actor'
    self.cri_name = name+' critc'
    self.tar_act_name = name+' target_actor'
    self.tar_cri_name = name+' target_critc'
    
    self.actor = ActorNet(n_actions = n_actions, dim_layer = layer_size)
    self.critic = CriticNet(n_actions,layer_size)

    self.target_actor = ActorNet(n_actions = n_actions, dim_layer = layer_size)
    self.target_critic = CriticNet(n_actions,layer_size)

    self.actor.compile(keras.optimizers.Adam(learning_rate=alpha))
    self.target_actor.compile(keras.optimizers.Adam(learning_rate=alpha))

    self.critic.compile(keras.optimizers.Adam(learning_rate=beta))
    self.target_critic.compile(keras.optimizers.Adam(learning_rate=beta))

    self.learnCounter = 0

  def Update_net_parameters(self,tau=None):
    if tau is None:
      tau = self.tau
    
    weights = []
    targets = self.target_actor.weights

    for i,weight in enumerate(self.actor.weights):
      weights.append(weight * tau + targets[i] *(1-tau))
    self.target_actor.set_weights(weights)

    weights = []
    targets = self.target_critic.weights  
    for i,weight in enumerate(self.critic.weights):
      weights.append(weight * tau + targets[i] *(1-tau))
    self.target_critic.set_weights(weights)

  def store_transition(self,state,action,reward,done):
    self.memory.store_trans(state,action,reward,done)

  def save_models(self):
    print('....saving models....')
    self.actor.save_weights(self.act_name)
    self.critic.save_weights(self.cri_name)
    self.target_actor.save_weights(self.tar_act_name)
    self.target_critic.save_weights(self.tar_cri_name)

    self.actor.save(self.act_name)

  def load_models(self):
    print('....loading models....')
    self.actor.load_weights(self.act_name)
    self.critic.load_weights(self.cri_name)
    self.target_actor.load_weights(self.tar_act_name)
    self.target_critic.load_weights(self.tar_cri_name)

  def choose_action(self,observation,evaluate = False):
    state = tf.convert_to_tensor([observation])
    probs = self.actor(state)
    action_probs = tfp.distributions.Categorical(probs = probs)
    action = action_probs.sample()
    action = action.numpy()[0]
    return action

  def learn(self,agents):
    
    if self.memory.mem_counter < self.batch_size:
      return
    
    self.learnCounter += 1
    agent_2 = agents[0]

    samples = self.memory.returnSample(self.batch_size)

    state_1,action_1,reward_1,new_state_1,done_1 = self.memory.sample_buffer(samples)
    states_1 = tf.convert_to_tensor(state_1)
    new_states_1 = tf.convert_to_tensor(new_state_1)
    rewards_1 = tf.convert_to_tensor(reward_1, dtype= tf.float32)
    actions_1 = tf.convert_to_tensor(action_1,dtype= tf.float32)

    state_2,action_2,reward_2,new_state_2,done_2 = agent_2.memory.sample_buffer(samples)
    states_2 = tf.convert_to_tensor(state_2)
    new_states_2 = tf.convert_to_tensor(new_state_2)
    rewards_2 = tf.convert_to_tensor(reward_2)
    actions_2 = tf.convert_to_tensor(action_2,dtype= tf.float32)


    with tf.GradientTape() as tape:
        target_actions_probs_1 = self.target_actor(new_states_1)
        target_probs = tfp.distributions.Categorical(probs = target_actions_probs_1)
        target_actions_1 = tf.transpose([target_probs.sample()])
    
        target_actions_probs_2 = agent_2.target_actor(new_states_2)
        target_probs = tfp.distributions.Categorical(probs = target_actions_probs_2)
        target_actions_2 = tf.transpose([target_probs.sample()])
      

        actions_1 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(actions_1), [0]))
        actions_2 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(actions_2), [0]))
        actions = tf.concat([actions_1,actions_2],axis = 1)
        actions = tf.cast(actions,tf.float32)

        new_states = tf.concat([new_states_1,new_states_2],axis = 1)
        target_actions = tf.concat([target_actions_1,target_actions_2], axis = 1)
        target_actions = tf.cast(target_actions,tf.float32)

      
        critic_value_ = tf.squeeze(self.target_critic((new_states,target_actions)),1)
        states= tf.concat([states_1,states_2],axis = 1)
     
        critic_value = tf.squeeze(self.critic((states,actions)),1)
        target = rewards_1 + self.gamma*critic_value_*(1-done_1)
        critic_loss = keras.losses.MSE(target,critic_value)
    
    params = self.target_critic.trainable_variables
    grads = tape.gradient(critic_loss,params)
    self.critic.optimizer.apply_gradients(zip(grads,params))


    with tf.GradientTape() as tape:
        new_policy_actions_probs = self.actor(states_1)
        new_probs = tfp.distributions.Categorical(probs = new_policy_actions_probs)
        new_actions_1 = tf.transpose([new_probs.sample()])

        new_actions_probs_2 = agent_2.actor(states_2)
        new_probs = tfp.distributions.Categorical(probs = new_actions_probs_2)
        new_actions_2 = tf.transpose([new_probs.sample()])

        new_actions = tf.concat([new_actions_1,new_actions_2],axis =1 )
        new_actions = tf.cast(new_actions,tf.float32)
        states = tf.concat([states_1,states_2],axis = 1)
          
        actor_loss = -self.critic((states,new_actions))
        actor_loss = actor_loss[0]
    
    params = self.critic.trainable_variables
    grads = tape.gradient(actor_loss,params)
    self.actor.optimizer.apply_gradients(zip(grads,params))

    if (self.learnCounter % 100 == 0):
      self.Update_net_parameters()
      self.learnCounter = 0

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp

class Agent_Opp:
  def __init__(self,input_dims,alpha = 0.01,beta =1e-5 , env = None ,
               gamma = 0.95, n_actions = 5, max_size = 10000, tau = 0.001,
               layer_size = 256, batch_size = 50,noise = 0.1, name = None):
    
    self.gamma = gamma
    self.tau = tau
    self.memory = Replay(max_size,n_actions,input_dims)
    self.batch_size = batch_size
    self.n_actions = n_actions
    self.act_name = name+' actor'
    self.cri_name = name+' critc'
    self.tar_act_name = name+' target_actor'
    self.tar_cri_name = name+' target_critc'
    
    self.actor = ActorNet(n_actions = n_actions, dim_layer = layer_size)
    self.critic = CriticNet(n_actions,layer_size)

    self.target_actor = ActorNet(n_actions = n_actions, dim_layer = layer_size)
    self.target_critic = CriticNet(n_actions,layer_size)

    self.actor.compile(keras.optimizers.Adam(learning_rate=alpha))
    self.target_actor.compile(keras.optimizers.Adam(learning_rate=alpha))

    self.critic.compile(keras.optimizers.Adam(learning_rate=beta))
    self.target_critic.compile(keras.optimizers.Adam(learning_rate=beta))

    self.learnCounter = 0

  def Update_net_parameters(self,tau=None):
    if tau is None:
      tau = self.tau
    
    weights = []
    targets = self.target_actor.weights

    for i,weight in enumerate(self.actor.weights):
      weights.append(weight * tau + targets[i] *(1-tau))
    self.target_actor.set_weights(weights)

    weights = []
    targets = self.target_critic.weights  
    for i,weight in enumerate(self.critic.weights):
      weights.append(weight * tau + targets[i] *(1-tau))
    self.target_critic.set_weights(weights)

  def store_transition(self,state,action,reward,done):
    self.memory.store_trans(state,action,reward,done)

  def save_models(self):
    print('....saving models....')
    self.actor.save_weights(self.act_name)
    self.critic.save_weights(self.cri_name)
    self.target_actor.save_weights(self.tar_act_name)
    self.target_critic.save_weights(self.tar_cri_name)

    self.actor.save(self.act_name)

  def load_models(self):
    print('....loading models....')
    self.actor.load_weights(self.act_name)
    self.critic.load_weights(self.cri_name)
    self.target_actor.load_weights(self.tar_act_name)
    self.target_critic.load_weights(self.tar_cri_name)

  def choose_action(self,observation,evaluate = False):
    state = tf.convert_to_tensor([observation])
    probs = self.actor(state)
    action_probs = tfp.distributions.Categorical(probs = probs)
    action = action_probs.sample()
    action = action.numpy()[0]
    return action

  def learn(self):
    if self.memory.mem_counter < self.batch_size:
      return

    self.learnCounter += 1

    samples = self.memory.returnSample(self.batch_size)

    state_1,action_1,reward_1,new_state_1,done_1 = self.memory.sample_buffer(samples)
    states_1 = tf.convert_to_tensor(state_1)
    new_states_1 = tf.convert_to_tensor(new_state_1)
    rewards_1 = tf.convert_to_tensor(reward_1, dtype= tf.float32)
    actions_1 = tf.convert_to_tensor(action_1,dtype= tf.float32)

    with tf.GradientTape() as tape:
        target_actions_probs_1 = self.target_actor(new_states_1)
        target_probs = tfp.distributions.Categorical(probs = target_actions_probs_1)
        target_actions_1 = tf.transpose([target_probs.sample()])  

        actions_1 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(actions_1), [0]))
        actions = tf.cast(actions_1,tf.float32)

        target_actions = tf.cast(target_actions_1,tf.float32)
        critic_value_ = tf.squeeze(self.target_critic((new_states_1,target_actions)),1)
        states = states_1
        critic_value = tf.squeeze(self.critic((states,actions)),1)
        target = rewards_1 + self.gamma*critic_value_*(1-done_1)
        critic_loss = keras.losses.MSE(target,critic_value)
    params = self.target_critic.trainable_variables
    grads = tape.gradient(critic_loss,params)
    self.critic.optimizer.apply_gradients(zip(grads,params))


    with tf.GradientTape() as tape:
        new_policy_actions_probs = self.actor(states_1)
        new_probs = tfp.distributions.Categorical(probs = new_policy_actions_probs)
        new_actions_1 = tf.transpose([new_probs.sample()])
        new_actions = tf.cast(new_actions_1,tf.float32)
        states = states_1
        actor_loss = -self.critic((states,new_actions))
        actor_loss = actor_loss[0]
    params = self.critic.trainable_variables
    grads = tape.gradient(actor_loss,params)
    self.actor.optimizer.apply_gradients(zip(grads,params))

    if (self.learnCounter % 100 == 0):
      self.Update_net_parameters()
      self.learnCounter = 0

In [ ]:
env.reset()
agent_net = {}
agent_list = []
for a in env.agents:
  n_actions = env.action_space(a).n
  input_dims = env.observation_space(a).shape[0]
  if(a == 'adversary_0'):
    agent_Opp = Agent_Opp(input_dims= input_dims ,alpha = 1e-5,env = env , n_actions = n_actions,name = a)
  else:
    agent =  Agent_Good(input_dims= input_dims ,alpha = 1e-5,env = env , n_actions = n_actions,name = a)
    agent_list.append(a)
    agent_net[a] = agent

In [ ]:
n_games = 2500
Good_best_score = -1000
Opp_best_score = -1000
Opp_score_history = []
Good_score_history = []
Opp_state_ = []
Good_state_ = []
load_checkpoint = False
if load_checkpoint:
  for a in agent_list:
    agent_net[a].load_models()
else:
  evaluate = False
for i in range(n_games):
  Opp_score = 0
  Good_score = 0
  env.reset(seed = 3)
  for agent in env.agent_iter():
    state,reward,done,trunc,info = env.last()
    if (agent == 'adversary_0'):
      if not (done or trunc):
        action = agent_Opp.choose_action(state,evaluate)
        env.step(action)
        Opp_score += reward
        agent_Opp.store_transition(state,action,reward,done)
        agent_Opp.learn() 
      else:
        env.step(None)
    else:
      if not (done or trunc):
        action = agent_net[agent].choose_action(state,evaluate)
        env.step(action)
        Good_score += reward
        for a in agent_list:
          if (a != agent):  
            agent_net[agent].learn([agent_net[a]])
      else:
        env.step(None)

  Good_score_history.append(Good_score)
  Opp_score_history.append(Opp_score)
  Good_avg_score = np.mean(Good_score_history[-100:])
  Opp_avg_score = np.mean(Opp_score_history[-100:])

  if Good_avg_score > Good_best_score:
    Good_best_score = Good_avg_score
    if not load_checkpoint:
      for a in agent_list:
        agent_net[a].save_models()

  if Opp_avg_score > Opp_best_score:
    Opp_best_score = Opp_avg_score
    if not load_checkpoint:
      agent_Opp.save_models()
  
  
  print('episode: ', i,'Opp average score: %.2f' % Opp_avg_score,'Good average score %.2f' % Good_avg_score)

....saving models....
....saving models....
....saving models....
episode:  0 Opp average score: -29.40 Good average score 30.01
....saving models....
episode:  1 Opp average score: -24.69 Good average score 21.92
episode:  2 Opp average score: -25.47 Good average score 21.40
....saving models....
episode:  3 Opp average score: -23.90 Good average score 22.42
....saving models....
episode:  4 Opp average score: -22.98 Good average score 15.56
....saving models....
episode:  5 Opp average score: -22.10 Good average score 14.01
episode:  6 Opp average score: -22.66 Good average score 15.64
episode:  7 Opp average score: -23.19 Good average score 15.14
episode:  8 Opp average score: -22.32 Good average score 12.59
....saving models....
episode:  9 Opp average score: -20.94 Good average score 11.49
....saving models....
episode:  10 Opp average score: -20.55 Good average score 10.31
episode:  11 Opp average score: -21.56 Good average score 11.50
episode:  12 Opp average score: -20.98 Good 

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

x = np.linspace(0,2*np.pi,100)
y = np.sin(x) + np.random.random(100) * 0.2
yhat = savgol_filter(Good_score_history, 101, 7) # window size 51, polynomial order 3


plt.plot(yhat, color='red')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

x = np.linspace(0,2*np.pi,100)
y = np.sin(x) + np.random.random(100) * 0.2
yhat = savgol_filter(Opp_score_history, 101, 7) # window size 51, polynomial order 3


plt.plot(yhat, color='red')
plt.show()

In [ ]:
print(Good_score_history)

[30.01420705024381, 13.819526775141483, 20.373937896911208, 25.459741095835636, -11.858763341676212, 6.243069707393786, 25.400770159777228, 11.697699702225673, -7.8674828072063265, 1.6404701473939105, -1.4836515523859832, 24.52481103677897, -10.443388837532053, -8.438620245545993, -0.5989757795356512, 5.3046852339935615, 23.711433176161222, 19.861206243878204, -10.84605914707455, -7.890506515187111, -1.7336986561061511, 37.089019555770264, 16.391800666692568, 15.926017180688923, -20.86760583095586, 19.40603433229639, -17.902891412324397, -2.469721360844458, 12.580421951893047, 11.820126849359141, -15.69315592697179, 2.176356515776476, 2.203749369681348, -3.6726598415329095, -14.235404424456036, -26.178486744372236, 1.8160863586499096, -7.7442553522714785, -0.08508320026594851, 23.531149758062497, 12.929896785247722, -9.825710335953719, 20.720245838127443, 2.7913199946740352, -5.829852184147594, -13.60057982634657, 5.9605060922921655, -4.298422156943168, 15.897060292873459, 25.514782111

In [ ]:
print(Opp_score_history)

[-29.401439576351372, -19.986130700789726, -27.013832531277632, -19.193681622711452, -19.326722919065006, -17.683331221977635, -26.030272963108988, -26.888000192421632, -15.366070565379474, -8.506007461469675, -16.627387182709512, -32.69045213249843, -13.980231072873686, -18.182559058557487, -16.82547751463674, -26.96061954573096, -19.716523796488275, -30.00377431238672, -19.288713941750874, -19.106772442054762, -26.432321622130534, -28.312020548610636, -25.012556083314703, -18.566944087760096, -16.354805502269635, -24.716455717920322, -22.60330996766384, -11.984120096520485, -28.146962529631928, -23.994624418593254, -12.54904271676862, -13.605846493882737, -11.324368305684219, -16.437417670674346, -16.28040589688185, -22.253743752574294, -13.172423096401358, -10.998274063820675, -16.33111763664804, -34.84489585140878, -24.436640018592016, -9.071707018598676, -32.29605205181508, -21.42451050906587, -25.203595706344366, -16.372588578333193, -21.374480065786972, -21.32358595981511, -24.4

In [ ]:
!zip -r /content/adversary_0_actor.zip /content/adversary_0_actor
!zip -r /content/agent_0_actor.zip /content/agent_0_actor
!zip -r /content/agent_1_actor.zip /content/agent_1_actor

  adding: content/adversary_0_actor/ (stored 0%)
  adding: content/adversary_0_actor/assets/ (stored 0%)
  adding: content/adversary_0_actor/keras_metadata.pb (deflated 80%)
  adding: content/adversary_0_actor/saved_model.pb (deflated 86%)
  adding: content/adversary_0_actor/variables/ (stored 0%)
  adding: content/adversary_0_actor/variables/variables.index (deflated 51%)
  adding: content/adversary_0_actor/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: content/agent_0_actor/ (stored 0%)
  adding: content/agent_0_actor/assets/ (stored 0%)
  adding: content/agent_0_actor/keras_metadata.pb (deflated 80%)
  adding: content/agent_0_actor/saved_model.pb (deflated 86%)
  adding: content/agent_0_actor/variables/ (stored 0%)
  adding: content/agent_0_actor/variables/variables.index (deflated 47%)
  adding: content/agent_0_actor/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: content/agent_1_actor/ (stored 0%)
  adding: content/agent_1_actor/assets/ (stored 0

In [ ]:
from google.colab import files

files.download('/content/agent_1_actor.zip')
files.download('/content/agent_0_actor.zip')
files.download('/content/adversary_0_actor.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>